# 受信時刻調整用ノートブック

## ライブラリのインポート

In [31]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm

from lib import Util

## 設定ファイルの読み込み

In [32]:
# 設定ファイルの読み込み
with open(f"{Util.get_root_dir()}/../config/config.json", "r") as f:
    config = json.load(f)

## 受信時刻調整アルゴリズム

In [33]:
class TimeAdjuster:
    """受信時刻補正を行うクラス"""
    def __init__(self, df_dict: dict):
        self.df_dict   = df_dict               # データフレームの辞書
        self.time_dict = self._get_time_dict() # 各データフレームのTime列（Index）のdict

    def _get_time_dict(self) -> dict:
        """各データフレームのTime列（Index）のdictを取得する"""
        time_dict = {}
        for key in self.df_dict.keys():
            time_dict[key] = np.array(self.df_dict[key].index)
        return time_dict

    def calc_idx_time_std(self, idx: int) -> float:
        """time_dictの任意の列"""
        idx_time_list = [float(self.time_dict[key][idx]) for key in self.time_dict.keys()]
        return np.nanstd(idx_time_list)

    def calc_idx_time_argmax(self, idx: int) -> int:
        """最大値の列を計算する"""
        idx_time_list = [float(self.time_dict[key][idx]) for key in self.time_dict.keys()]
        return np.nanargmax(idx_time_list)

    def shift_idx_time(self, key: str, idx: int) -> None:
        """time_dictの任意の列をidx行から1つシフトする"""
        shifted             = np.insert(self.time_dict[key], idx+1, self.time_dict[key][idx])
        shifted[idx]        = np.nan
        self.time_dict[key] = shifted


## 受信時刻調整

In [34]:
# 共通ファイルを取得
common_file = Util.get_common_files(path_list=[f"{Util.get_root_dir()}/../data/preprocessed-data/{field_device}/amp/" for field_device in config["FieldDevice"]["Pcap"]])

In [ ]:
ALPHA = 0.07  # 補正係数

# 各ファイルに対して受信時間調整アルゴリズムを適用
for file_name in tqdm(common_file):
    df_dict = {}
    for field_device in sorted(config["FieldDevice"]["Pcap"]):
        # ファイルのパスを取得
        file_path = f"{Util.get_root_dir()}/../data/preprocessed-data/{field_device}/amp/{file_name}"
        # データを読み込み
        df_dict[field_device] = pd.read_csv(file_path, index_col=0)

    # 受信時間調整アルゴリズムを適用
    ta = TimeAdjuster(df_dict=df_dict)
    for idx in range(len(ta.time_dict[list(ta.time_dict.keys())[0]])):
        if ta.calc_idx_time_std(idx) > ALPHA:
            argmax = ta.calc_idx_time_argmax(idx)
            key    = list(ta.time_dict.keys())[argmax]
            ta.shift_idx_time(key=key, idx=idx)

    # 調整後のデータを保存
    for key in ta.df_dict.keys():
        Util.create_path(f"{Util.get_root_dir()}/tmp/{key}/amp/")
        pd.DataFrame(ta.time_dict[key], columns=["Time"]).to_csv(f"{Util.get_root_dir()}/tmp/{key}/amp/{file_name}")

100%|██████████| 4/4 [00:00<00:00, 50.92it/s]
